### Задание
Имеются данные adult.csv (см. в материалах к занятию).  
Целевой переменной является уровень дохода income (крайний правый столбец).  
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html  
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.  
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.  

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

from sklearn.svm import SVC

In [24]:
data = pd.read_csv('1\\adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [26]:
data['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [27]:
# признаки 
selectedColumns = data [['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
                         'marital-status', 'occupation', 'relationship', 'race', 'gender',
                         'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
                         'income']]

# трансформируем категориальные столбцы в значения 0 и 1
X = pd.get_dummies(selectedColumns, columns = 
                    ['workclass', 'education', 'marital-status', 'occupation',
                     'relationship', 'race', 'gender', 'native-country'])

# income целевая переменная, удаляем из Х
del X['income']

X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,30,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [28]:
le = LabelEncoder()

In [29]:
le.fit(data['income'])

LabelEncoder()

In [30]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [32]:
Y = pd.Series(data = le.transform(data['income']))
Y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [33]:
model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [36]:
# обучаем 
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [37]:
predictions[:5]

array([0, 0, 1, 0, 1])

In [39]:
le.inverse_transform(predictions[:5]) # посмотрим как выглядят наши предсказания

array(['<=50K', '<=50K', '>50K', '<=50K', '>50K'], dtype=object)

In [40]:
model.predict_proba(X_test)

array([[0.98642455, 0.01357545],
       [0.90248731, 0.09751269],
       [0.06125033, 0.93874967],
       ...,
       [0.97750439, 0.02249561],
       [0.70038098, 0.29961902],
       [0.982159  , 0.017841  ]])

In [41]:
model.score(X_train, y_train)

0.8517944368071602

In [42]:
model.score(X_test, y_test)

0.8572306012420665

### Метод опорных векторов. SVM.

In [43]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [44]:
clf.score(X_train, y_train) 

0.8612711690894732

In [45]:
clf.score(X_test, y_test)

0.8551832389271822